## Import

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns

from CinePred.data.importing import import_data
from CinePred.data.preprocessing import *
from CinePred.data.featuring import *
from CinePred.pipeline import *

import cpi
import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
df = import_data('../raw_data/IMDb movies.csv')

# CLEANING
df = keep_columns(df,column_names=[
    'imdb_title_id', 'title', 'year', 'date_published', 'genre',
    'duration', 'country', 'director', 'writer', 'production_company',
    'actors', 'budget', 'worlwide_gross_income'
])
df = remove_na_rows(df)
df['year'] = convert_to_int(df[['year']])
df['duration'] = convert_to_int(df[['duration']])
df['worlwide_gross_income'] = convert_income(df[['worlwide_gross_income']])
df['budget'] = convert_budget_column(df[['budget']], path='../raw_data/currencies.csv')
df['date_published'] = convert_to_date(df[['date_published']])
df = df.sort_values(by='date_published')

In [23]:
def inflate_column_1(data, column):
    data["year_2"] = data["year"].apply(lambda x: 2018 if x > 2018 else x)
    new = column + "_inflated"
    data[new] = data.apply(lambda x: cpi.inflate(x[column], x["year_2"]),
                           axis=1)
    return data

In [24]:
print('----- add column inflated-----')
df = inflate_column_1(df, "budget")

print('----- add income inflated-----')
df = inflate_column_1(df, "worlwide_gross_income")

# print('----- add_inflation_budget -----')
# df["inflation_budget"] = add_inflation_budget(df[["budget"]])

----- add column inflated-----
----- add income inflated-----


In [25]:
df.head(5)

,imdb_title_id,title,year,date_published,genre,duration,country,director,writer,production_company,actors,budget,worlwide_gross_income,year_2,budget_inflated,worlwide_gross_income_inflated
0,tt0010323,Il gabinetto del dottor Caligari,1920,1920-02-27,"Fantasy, Horror, Mystery",76,Germany,Robert Wiene,"Carl Mayer, Hans Janowitz",Decla-Bioscop AG,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",18000,8811,1920,2.259963e+05,1.106252e+05
1,tt0011440,Markens grøde,1921,1921-12-02,Drama,107,Norway,Gunnar Sommerfeldt,"Knut Hamsun, Gunnar Sommerfeldt",Christiana Film,"Amund Rydland, Karen Poulsen, Ragna Wettergree...",27887,4272,1921,3.912079e+05,5.992900e+04
2,tt0012190,I quattro cavalieri dell'Apocalisse,1921,1923-04-16,"Drama, Romance, War",150,USA,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",800000,9183673,1921,1.122266e+07,1.288315e+08
3,tt0012349,Il monello,1921,1923-11-26,"Comedy, Drama, Family",68,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Carl Miller, Edna Purviance, Jackie Coogan, Ch...",250000,26916,1921,3.507081e+06,3.775864e+05
5,tt0015864,La febbre dell'oro,1925,1925-10-23,"Adventure, Comedy, Drama",95,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Mack Swain, Tom Murray, Henry...",923000,26916,1925,1.324410e+07,3.862169e+05


In [8]:
X = df[[
        'budget', 'genre', 'duration', 'year', 'date_published',
        'production_company', 'writer', 'director'
    ]]
y = df['worlwide_gross_income']
y = np.log(y) / np.log(10)

In [9]:
pipeline = create_pipeline()

In [ ]:
mae = fit_and_score(pipeline, X, y)